In [74]:
# 비동기적 웹 : 페이지 내용은 변경되나 URL은 그대로인 웹페이지
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime

In [5]:
princ_url = 'http://fchart.stock.naver.com/sise.nhn?symbol=005930&timeframe=day&count=1500&requestType=0'
params = {
    'symbol': '005930',
    'timeframe': 'day',
    'count' : 1500,
    'requestType' : 0,
    }
resp = requests.get(princ_url, params=params)

In [54]:
price_data = BeautifulSoup(resp.text, 'lxml')
infor = price_data.find('chartdata')
infor['name']

'삼성전자'

In [14]:
item_list = price_data.find_all('item')
item_list[0]

<item data="20131220|28260|28540|28260|28540|168148"></item>

In [15]:
temp = item_list[0]['data']
temp.split('|')

['20131220', '28260', '28540', '28260', '28540', '168148']

In [18]:
date_list = []
price_list = []
for item in item_list : 
    temp_data = item['data']
    stock_data = temp_data.split("|")
    date_list.append(stock_data[0])
    price_list.append(stock_data[4])

price_df = pd.DataFrame({
    '종가' : price_list,}
,index = date_list)

display(price_df.head(10))

,종가
20131220,28540
20131223,28660
20131224,28300
20131226,28160
20131227,27920
20131230,27440
20140102,26180
20140103,25920
20140106,26140
20140107,26080


In [65]:
def make_price_dataframe(code, timeframe, count) : 
    princ_url = 'http://fchart.stock.naver.com/sise.nhn?'
    params = {
        'symbol': code,
        'timeframe': timeframe,
        'count' : count,
        'requestType' : 0,
        }
    resp = requests.get(princ_url, params=params)
    price_data = BeautifulSoup(resp.text, 'lxml')
    infor = price_data.find('chartdata')
    corp_name = infor['name']
    item_list = price_data.find_all('item')
    
    date_list = []
    price_list = []
    for item in item_list : 
        temp_data = item['data']
        stock_data = temp_data.split("|")
        date_list.append(stock_data[0])
        price_list.append(stock_data[4])

    price_df = pd.DataFrame({
        corp_name : price_list,}
        ,index = date_list)

    return price_df

In [66]:
make_price_dataframe('005380', 'day', '1500').head(10)

,현대차
20131220,224500
20131223,228500
20131224,226000
20131226,225500
20131227,229500
20131230,236500
20140102,224500
20140103,224000
20140106,228500
20140107,229000


concat은 주로 같은 칼럼명을 가진 데이터프레임을 위 아래로 합칠 때 사용하는 데, 

지금의 경우 칼러명은 서로 다르고 인덱스(row)가 날짜로 같기 때문에 옆으로 붙여야함 >> merge 사용

In [68]:
firmcode_list = ['005930', '005380', '035420', '003550', '034730']

for num, code in enumerate(firmcode_list) : 
    price_df = make_price_dataframe(code, 'day', '1500')
    if num == 0 :
        total_price = price_df
    else : 
        total_price = pd.merge(total_price,price_df,how='outer', right_index=True, left_index=True)

display(total_price.head(10))

,삼성전자,현대차,NAVER,LG,SK
20131220,28540,224500,147409,62100,130000
20131223,28660,228500,148210,62800,129000
20131224,28300,226000,147009,63000,133000
20131226,28160,225500,143003,63700,133500
20131227,27920,229500,147009,63800,132000
20131230,27440,236500,145006,64000,135000
20140102,26180,224500,145206,62700,131500
20140103,25920,224000,139999,61700,128500
20140106,26140,228500,139999,61000,128500
20140107,26080,229000,140199,61000,129500


In [72]:
file_path = 'C:/Users/elris/Dropbox/Data Scientist/Python/엑셀 데이터/KOSPI200.xlsx'

code_data = pd.read_excel(file_path)
code_data = code_data[['종목코드','기업명']]

def make_code (x) :
    x = str(x)
    
    return '0'*(6-len(x)) + x

code_data['종목코드'] = code_data['종목코드'].apply(make_code)

In [73]:
for num, code in enumerate(code_data['종목코드']) : 
    try : 
        print(num, code)
        time.sleep(1)
        try : 
            price_df = make_price_dataframe(code, 'day', '1500')
        except requests.exceptions.Timeout : 
            time.sleep(30)
            price_df = make_price_dataframe(code, 'day', '1500')
        if num == 0 :
            total_price = price_df
        else : 
            total_price = pd.merge(total_price,price_df,how='outer', right_index=True, left_index=True)
    except ValueError:
        continue
    except KeyError:
        continue

0 005930
1 000660
2 207940
3 035420
4 005380
5 051910
6 012330
7 068270
8 028260
9 006400
10 051900
11 005490
12 055550
13 017670
14 105560
15 000270
16 034730
17 015760
18 018260
19 036570
20 035720
21 032830
22 033780
23 003550
24 096770
25 090430
26 066570
27 086790
28 000810
29 009150
30 010950
31 009540
32 251270
33 316140
34 010130
35 011170
36 030200
37 021240
38 024110
39 002790
40 035250
41 032640
42 018880
43 086280
44 034220
45 006800
46 267250
47 078930
48 029780
49 000720
50 010140
51 271560
52 004020
53 004990
54 071050
55 011070
56 161390
57 097950
58 012750
59 008770
60 128940
61 028050
62 023530
63 000120
64 005940
65 016360
66 139480
67 007070
68 241560
69 036460
70 005830
71 047810
72 000100
73 282330
74 000210
75 009830
76 042660
77 081660
78 004170
79 003410
80 030000
81 001040
82 006360
83 020150
84 008560
85 003490
86 138930
87 000080
88 008930
89 011780
90 028670
91 047050
92 011790
93 001450
94 069960
95 007310
96 002380
97 047040
98 088350
99 012450
100 051600

In [77]:
total_price.head(20)

,삼성전자,SK하이닉스,삼성바이오로직스,NAVER,현대차,LG화학,현대모비스,셀트리온,삼성물산,삼성SDI,...,한국쉘석유,남해화학,유니드,JW홀딩스,고려제강,쌍용차,남양유업,동양,현대리바트,효성중공업
20131220,28540,35300,NaN,147409,224500,294000,281500,31543,NaN,162000,...,479000,6670,41984,2096,28352,6850,930000,1540,11600,NaN
20131223,28660,37000,NaN,148210,228500,291000,285000,32511,NaN,163500,...,483000,6810,42726,2092,28539,6800,918000,1550,11700,NaN
20131224,28300,36750,NaN,147009,226000,294500,287000,32511,NaN,161000,...,487000,6870,43542,2067,28314,6820,935000,1540,11800,NaN
20131226,28160,36000,NaN,143003,225500,296000,288500,32006,NaN,160000,...,491000,6790,43690,2117,28314,6790,938000,1500,11550,NaN
20131227,27920,36650,NaN,147009,229500,295500,289000,33045,NaN,162000,...,471500,6740,44209,2192,28051,7000,925000,1455,11650,NaN
20131230,27440,36800,NaN,145006,236500,299500,293500,33175,NaN,162000,...,468500,6900,43913,2174,28051,7550,940000,1450,12300,NaN
20140102,26180,35550,NaN,145206,224500,292000,279000,33346,NaN,157500,...,458000,6850,42355,2187,27639,7150,933000,1470,12900,NaN
20140103,25920,36300,NaN,139999,224000,286000,276000,35380,NaN,153500,...,436500,6840,42058,2212,27338,7290,931000,1460,12950,NaN
20140106,26140,37650,NaN,139999,228500,284500,279500,35034,NaN,148000,...,450500,6870,41539,2205,27188,7110,937000,1460,12850,NaN
20140107,26080,37800,NaN,140199,229000,277500,277500,37456,NaN,151000,...,443000,7140,41316,2212,27338,7120,939000,1455,13200,NaN


In [79]:
total_price.index = pd.to_datetime(total_price.index)
total_price.to_excel('C:/Users/elris/Dropbox/Data Scientist/Python/엑셀 데이터/가격데이터.xlsx')